In [ ]:
from ib_insync import *
import requests
import pandas as pd
import schedule
import time
import datetime;
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
# https://stackoverflow.com/questions/44933518/how-to-remove-runtimewarning-errors-from-code

util.startLoop()  # only use in interactive environments (i.e. Jupyter Notebooks)
# https://algotrading101.com/learn/ib_insync-interactive-brokers-api-guide/
# https://www.reddit.com/r/interactivebrokers/comments/1cccg0t/ib_insync_reqhistoricaldata_with_keepuptodatetrue/
# https://github.com/erdewit/ib_insync/issues/653

In [ ]:
#WORKING
counter_of_numbers = 0
ib = IB()
ib.connect(host='127.0.0.1', port=7496, clientId=0)
# https://www.elitetrader.com/et/threads/cant-connect-to-api-interactive-brokers.361205/
# needed to change socket port to 7496 and uncheck the Read-Only API box

# Data Dumps

ib.positions()

Position(

account='U12848716', 

contract=Stock(conId=699197691, symbol='APDN', exchange='NASDAQ', currency='USD', localSymbol='APDN', tradingClass='SCM'), 

position=6.0, avgCost=0.20738335          ),

So...

Let's try:

positions.contract.symbol
positions.position

/*
for _ in range(len(posits)): 
   print(posits[_].avgCost)
*/   

ib.fills()

Fill(

contract=Stock(conId=699197691, symbol='APDN', exchange='IBKRATS', currency='USD', localSymbol='APDN', tradingClass='SCM'), 

execution=Execution(execId='0000ddc2.67812bcb.01.01', 
time=datetime.datetime(2025, 1, 10, 17, 13, 37, tzinfo=datetime.timezone.utc), 
acctNumber='U12848716', exchange='IBKRATS', side='BOT', 
shares=1.0, price=0.199, 
permId=1655614144, clientId=0, orderId=3, 
liquidation=0, cumQty=1.0, avgPrice=0.199, 
orderRef='', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=2), 
    
commissionReport=CommissionReport(execId='0000ddc2.67812bcb.01.01', 
commission=0.002036, currency='USD', realizedPNL=0.0, yield_=0.0, yieldRedemptionDate=0), 
time=datetime.datetime(2025, 1, 10, 17, 13, 37, 297013, tzinfo=datetime.timezone.utc)           ),

So...

Let's try:

fill.contract.symbol

fill.execution.shares
fill.execution.price
fill.execution.avgPrice

ib.trades()

Trade(
    
contract=Stock(conId=670777621, symbol='SVMH', right='?', exchange='SMART', currency='USD', localSymbol='SVMH', 
tradingClass='NMS'), 
    
order=Order(permId=241210818, action='SELL', totalQuantity=1.0, orderType='LMT', lmtPrice=0.07, auxPrice=0.0, 
tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', 
referencePriceType=0, account='U12848716', clearingIntent='IB', cashQty=0.0, dontUseAutoPriceForHedge=True, 
autoCancelDate='20250630 17:00:00 EST', filledQuantity=0.0, refFuturesConId=2147483647, 
shareholder='Not an insider or substantial shareholder'), 
    
orderStatus=OrderStatus(orderId=0, status='Cancelled', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, 
parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[], advancedError=''             ),


So...

Let's try:

trade.contract.symbol

trade.order.totalQuantity
trade.order.orderType
trade.order.lmtPrice
trade.order.auxPrice
trade.order.filledQuantity

trade.orderStatus.filled
trade.orderStatus.remaining
trade.orderStatus.avgFillPrice

# Test January 13 - 17

We are looking for:

1) Can we use a ForLoop to register several (2+) orders that may all need to be enacted at 10 AM (theoretically possible)...
   This probably is easy since we are manually creating each order consecutively
2) Test whether the several orders will all have fillEvents following them and all executing on their appropriate times...
3) Have the df tracking them que up and que off rows of data...

In [ ]:
pending_stop_loss_orders = pd.DataFrame() # define and create one time
pos = ib.positions()

In [ ]:
def order_status(trade):
    if trade.orderStatus.status == 'Filled':
        fill = trade.fills[-1]
        for index, row in pending_stop_loss_orders.iterrows():
            if(row['TK'] == fill.contract.symbol and row['QT'] == fill.execution.shares):
                
                contract = Stock('NYMT', 'SMART', 'USD')
                stopOrder = StopOrder('BUY', fill.execution.shares, row['SLT'])
                stopOrderTrade = ib.placeOrder(contract, stopOrder)
                
                pending_stop_loss_orders.drop(index, inplace=True)   # no longer useful to keep (does this work?)
                
                
# test orders with cascading percentages

percentages = [1.01, 0.99, 1.03, 0.97, 1.05, 0.95]

apdn_contract = Stock('APDN', 'SMART', 'USD')
ib.qualifyContracts(apdn_contract)
data = ib.reqMktData(apdn_contract)

for percs in percentages:
    contract = Stock("APDN", 'SMART', 'USD')
    limitOrder = LimitOrder('SELL', 1, data.marketPrice())
    limitOrderTrade = ib.placeOrder(contract, limitOrder)
        
    # CREATE A DATAFRAME THAT TRACKS STOP LIMIT ORDERS MADE ALONG WITH THEIR STOP LOSS TARGETS
    new_row = pd.DataFrame({"TK": ["APDN"], "QT": [1], "SLT": [data.marketPrice() * 1.25]})
    pending_stop_loss_orders = pd.concat([pending_stop_loss_orders, new_row], ignore_index=True)
    
    limitOrderTrade.fillEvent += order_status

In [ ]:
if this works, determine how to track 5 minutes spans and remove all orders on a symbol and replace with highest Sl order for all shares owned..

In [ ]:
def order_status(trade):
    if trade.orderStatus.status == 'Filled':
        fill = trade.fills[-1]
        for index, row in pending_stop_loss_orders.iterrows():
            if(row['Ticker'] == fill.contract.symbol and row['Quantity'] == fill.execution.shares):
                
                contract = Stock(fill.contract.symbol, 'SMART', 'USD')
                stopOrder = StopOrder('SELL', fill.execution.shares, row['SLTarget'])
                stopOrderTrade = ib.placeOrder(contract, stopOrder)
                
                pending_stop_loss_orders.drop(row)   # no longer useful to keep (does this work?)
                break
                

positions = ib.positions()

for pos in positions(
    
     
                
    # create the contract detailing
    
    # classib_insync.order.MarketOrder(action, totalQuantity, **kwargs)
    # classib_insync.order.LimitOrder(action, totalQuantity, lmtPrice, **kwargs)
    # classib_insync.order.StopOrder(action, totalQuantity, stopPrice, **kwargs)
    # classib_insync.order.StopLimitOrder(action, totalQuantity, lmtPrice, stopPrice, **kwargs)
    
    if(pos.contract.symbol != "NUVOQ"):
        contract = Stock(pos.contract.symbol, 'SMART', 'USD')
        limitOrder = LimitOrder('SELL', 1, ISP)    
    
        # CREATE A DATAFRAME THAT TRACKS STOP LIMIT ORDERS MADE ALONG WITH THEIR STOP LOSS TARGETS
        new_row = pd.DataFrame({"TK": [pos.contract.symbol], "QT": [1], "SLT": [SLT]})
        pending_stop_loss_orders = pd.concat([pending_stop_loss_orders, new_row], ignore_index=True)
    
    #limitOrderTrade = ib.placeOrder(contract, limitOrder)
    #trade_details = limitOrderTrade    

    #limitOrderTrade.fillEvent += order_status
)    